# 3.8 Nonlinear minimization problems
We consider problems of the form

$$\text{find } u \in V \text{ s.t. } E(u) \leq E(v) \quad \forall~  v \in V.$$

In [1]:
import netgen.gui
%gui tk
from ngsolve import *

## Scalar minimization problems
As a first example we take $V = H^1_0$ and 

$$
E(u) = \int_{\Omega} \vert \nabla u \vert^2 + u^4 - fu ~dx.
$$

The minimization is equivalent to solving the nonlinear PDE:

$$
 - \Delta u + 4 u^3 = f \text{ in } \Omega
$$

We solve the PDE with a Newton iteration.

In [2]:
from netgen.geom2d import unit_square

mesh = Mesh (unit_square.GenerateMesh(maxh=0.2))
V = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = V.TrialFunction()

To solve the problem we use the `SymbolicEnergy` integrator. Based on the symbolic description of the energy functional, it is able to 

* evaluate the energy functional (`Energy`)

$$E(u) \qquad (E:V \to \mathbb{R})$$

* compute the Gateau derivative for a given $u$ (`Apply`):

$$A(u)(v) = E'(u)(v) \qquad (A(u): V \to \mathbb{R})$$

* compute the second derivative (`AssembleLinearization`)

$$(\delta A)(w)(u,v) \qquad (\delta A(w): V\times V \to \mathbb{R})$$

In [3]:
a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy ( grad(u)*grad(u) + u**4-u )

Equivalent to:
```
a += SymbolicBFI( 2 * grad(u) * grad(v) + 4*u*u*u*v - 1 * v)
``` 
(which has the same form as the problems in [the nonlinear example](../unit-3.7-nonlinear/nonlinear.ipynb))

We recall the Newton iteration (cf. [unit-3.7](../unit-3.7-nonlinear/nonlinear.ipynb) ) we make the loop:

- Given an initial guess $u^0$
- loop over $i=0,..$ until convergence:
  - Compute linearization: $A u^i + \delta A(u^i) \Delta u^{i} = 0 $:
    - $f^i = A u^i$ 
    - $B^i = \delta A(u^i)$ 
    - Solve $B^i \Delta u^i = -f^i$
  - Update $u^{i+1} = u^i + \Delta u^{i}$
  - Evaluate stopping criteria
- Evaluate $E(u^{i+1})$

As a stopping criteria we take $\langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.  

In [4]:
def SolveNonlinearMinProblem(a,gfu,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(maxits):
        print ("Newton iteration {:3}".format(it),end="")
        print ("energy = {:16}".format(a.Energy(gfu.vec)),end="")
    
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        du.data = inv * res
    
        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
        Redraw(blocking=True)

In [5]:
gfu = GridFunction (V)
gfu.vec[:] = 0
Draw(gfu,mesh,"u")

SolveNonlinearMinProblem(a,gfu)

print ("energy = ", a.Energy(gfu.vec))
    

Newton iteration   0energy =              0.0<A u 0 , A u 0 >_{-1}^0.5 =  0.1325595815712792
Newton iteration   1energy = -0.008785677986915054<A u 1 , A u 1 >_{-1}^0.5 =  1.1107597333627496e-05
Newton iteration   2energy = -0.008785678048604416<A u 2 , A u 2 >_{-1}^0.5 =  2.807506601278265e-13
Newton iteration   3energy = -0.008785678048604414<A u 3 , A u 3 >_{-1}^0.5 =  3.069514454612314e-17
energy =  -0.008785678048604414


## Nonlinear elasticity

We consider a beam which is fixed on one side and is subject to gravity only. We assume a Neo-Hookean hyperelastic material. The model is a nonlinear minimization problem with 

$$
  E(v) := \int_{\Omega} \frac{\mu}{2} ( \operatorname{tr}(F^T F-I)+\frac{2 \mu}{\lambda} \operatorname{det}(F^T F)^{\frac{\lambda}{2\mu}} - 1) - \gamma ~ (f,v) ~~ dx
$$

where $\mu$ and $\lambda$ are the Lamé parameters and $F = I + D v$ where $v: \Omega \to \mathbb{R}^2$ is the sought for displacement.

In [6]:
import netgen.geom2d as geom2d
from ngsolve import *

geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))


In [7]:
# E module and poisson number:
E, nu = 210, 0.2
# Lamé constants:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

V = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
u  = V.TrialFunction()

#gravity:
force = CoefficientFunction( (0,-1) )

In [8]:
def Pow(a, b):
    return exp (log(a)*b)
    
def NeoHook (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C), -lam/2/mu) - 1)

I = Id(mesh.dim)
F = I + u.Deriv()   # attention: row .. component, col .. derivative
C = F * F.trans  

factor = Parameter(1.0)

a = BilinearForm(V, symmetric=False)
a += SymbolicEnergy(  NeoHook (C).Compile() )
a += SymbolicEnergy(  (-factor * InnerProduct(force,u) ).Compile() )

We want to solve the minimization problem for $\gamma = 5$. Due to the high nonlinearity in the problem, the Newton iteration will not convergence with any initial guess. We approach the case $\gamma = 5$ by solving problems with $\gamma = i/10$ for $i=1,..,50$ and taking the solution of the previous problem as an initial guess.

In [9]:
gfu = GridFunction(V)
gfu.vec[:] = 0

Draw (gfu, mesh, "u")
SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

for loadstep in range(50):
    print ("loadstep", loadstep)
    factor.Set ((loadstep+1)/10)
    SolveNonlinearMinProblem(a,gfu)
    Redraw()

loadstep 0
Newton iteration   0energy = 8.749999999999996<A u 0 , A u 0 >_{-1}^0.5 =  0.016678457857220372
Newton iteration   1energy = 8.750132589925284<A u 1 , A u 1 >_{-1}^0.5 =  0.023333465280071634
Newton iteration   2energy = 8.749861149642193<A u 2 , A u 2 >_{-1}^0.5 =  0.000102324871854834
Newton iteration   3energy = 8.749861144406996<A u 3 , A u 3 >_{-1}^0.5 =  5.1656126548647754e-08
Newton iteration   4energy = 8.749861144407008<A u 4 , A u 4 >_{-1}^0.5 =  2.7534833076896285e-13
Newton iteration   5energy = 8.749861144407008<A u 5 , A u 5 >_{-1}^0.5 =  7.942625526684423e-16
loadstep 1
Newton iteration   0energy = 8.749583890792827<A u 0 , A u 0 >_{-1}^0.5 =  0.016596122310545317
Newton iteration   1energy = 8.749710095753294<A u 1 , A u 1 >_{-1}^0.5 =  0.022958368235712307
Newton iteration   2energy = 8.74944729855201<A u 2 , A u 2 >_{-1}^0.5 =  0.00014216311244477864
Newton iteration   3energy = 8.749447288552698<A u 3 , A u 3 >_{-1}^0.5 =  2.3257814734367656e-06
Newton ite

Newton iteration   0energy = 8.722483105125077<A u 0 , A u 0 >_{-1}^0.5 =  0.009507271533554363
Newton iteration   1energy = 8.722445738804593<A u 1 , A u 1 >_{-1}^0.5 =  0.003623338365125314
Newton iteration   2energy = 8.722439169043259<A u 2 , A u 2 >_{-1}^0.5 =  8.084105831464236e-05
Newton iteration   3energy = 8.722439165775617<A u 3 , A u 3 >_{-1}^0.5 =  3.9657874014284555e-07
Newton iteration   4energy = 8.72243916577554<A u 4 , A u 4 >_{-1}^0.5 =  1.5322265915943703e-13
Newton iteration   5energy = 8.72243916577554<A u 5 , A u 5 >_{-1}^0.5 =  7.020924862732914e-15
loadstep 16
Newton iteration   0energy = 8.719585981725038<A u 0 , A u 0 >_{-1}^0.5 =  0.009114276766163665
Newton iteration   1energy = 8.719550495831808<A u 1 , A u 1 >_{-1}^0.5 =  0.003137848417803922
Newton iteration   2energy = 8.719545568051617<A u 2 , A u 2 >_{-1}^0.5 =  6.561021540824207e-05
Newton iteration   3energy = 8.719545565899164<A u 3 , A u 3 >_{-1}^0.5 =  2.525155744874241e-07
Newton iteration   4en

Newton iteration   3energy = 8.66570038427571<A u 3 , A u 3 >_{-1}^0.5 =  2.6056777347312574e-10
Newton iteration   4energy = 8.665700384275706<A u 4 , A u 4 >_{-1}^0.5 =  9.019629514694351e-15
loadstep 33
Newton iteration   0energy = 8.66202518273859<A u 0 , A u 0 >_{-1}^0.5 =  0.005321865247746466
Newton iteration   1energy = 8.662011341718637<A u 1 , A u 1 >_{-1}^0.5 =  0.0004532612188666138
Newton iteration   2energy = 8.662011238905643<A u 2 , A u 2 >_{-1}^0.5 =  2.1962280959714483e-06
Newton iteration   3energy = 8.662011238903226<A u 3 , A u 3 >_{-1}^0.5 =  1.7877630682069847e-10
Newton iteration   4energy = 8.662011238903228<A u 4 , A u 4 >_{-1}^0.5 =  7.564973133263078e-15
loadstep 34
Newton iteration   0energy = 8.658308362786434<A u 0 , A u 0 >_{-1}^0.5 =  0.005200339855866979
Newton iteration   1energy = 8.658295128322484<A u 1 , A u 1 >_{-1}^0.5 =  0.000415104764483889
Newton iteration   2energy = 8.658295042094178<A u 2 , A u 2 >_{-1}^0.5 =  1.8467124404900584e-06
Newton 

## Allen-Cahn equation

The Allen-Cahn equations describe the process of phase separation and is the ($L^2$) gradient-flow equation to the energy
$$
  E(v) = \int_{\Omega} \varepsilon \vert \nabla v \vert^2~+~v^2(1-v^2) ~ dx
$$

i.e. the solution to the Allen-Cahn equation solves

$$
\partial_t u = \frac{\delta E}{\delta u}
$$

The quantity $u$ is an indicator for a phase where $-1$ refers to one phase and $1$ to another phase. 

The equation has two driving forces:

- $u$ is pulled into one of the two minima ($-1$ and $1$) of the nonlinear term $u^2(1-u^2)$ (separation of the phases)
- the diffusion term scaled with $\varepsilon$ enforces a smooth transition between the two phases. $\varepsilon$ determines the size of the transition layer

We use the "SymbolicEnergy" feature to formulate the energy minimization problem and combine it with an implicit Euler discretization:

$$
 M u^{n+1} - M u^n = \Delta t \underbrace{\frac{\delta E}{\delta u}}_{=:A(u)} (u^{n+1})
$$

which we can interpreted as a nonlinear minimization problem again with the energy

$$
  E^{IE}(v) = \int_{\Omega} \frac{\varepsilon}{2} \vert \nabla v \vert^2~+~v^2(1-v^2) + \frac{1}{2\Delta t} \vert v - u^n \vert^2 ~ dx
$$

To solve the nonlinear equation at every time step we again rely on Newton's method.

In [10]:
from ngsolve import *

from netgen.geom2d import *


periodic = SplineGeometry()
pnts = [ (0,0), (1,0), (1,1), (0,1) ]
pnums = [periodic.AppendPoint(*p) for p in pnts]

lright = periodic.Append ( ["line", pnums[0], pnums[1]],bc="periodic")
btop = periodic.Append ( ["line", pnums[1], pnums[2]], bc="periodic")
periodic.Append ( ["line", pnums[3], pnums[2]], leftdomain=0, rightdomain=1, copy=lright, bc="periodic")
periodic.Append ( ["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, copy=btop, bc="periodic")

mesh = Mesh (periodic.GenerateMesh(maxh=0.2))

V = Periodic(H1(mesh, order=4, dirichlet=[]))

u = V.TrialFunction()

eps = 4e-3
dt = 1e-1

gfu = GridFunction(V)
gfuold = GridFunction(V)

a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy (eps/2*grad(u)*grad(u)
                     + ((1-u*u)*(1-u*u))
                     + 0.5/dt*(u-gfuold)*(u-gfuold))

In [11]:
from math import pi
gfu = GridFunction(V)
gfu.Set(sin(2*pi*x))
#gfu.Set(sin(1e8*x)) #<- essentially a random function
Draw(gfu,mesh,"u")
SetVisualization (deformation=False)
t = 0

In [12]:
for timestep in range(50):
    gfuold.vec.data = gfu.vec
    SolveNonlinearMinProblem(a,gfu)
    Redraw() 
    t += dt
    print("t = ", t)

Newton iteration   0energy = 0.41450268691109665<A u 0 , A u 0 >_{-1}^0.5 =  0.2809232585283059
Newton iteration   1energy = 0.37690029538736747<A u 1 , A u 1 >_{-1}^0.5 =  0.019176192659565692
Newton iteration   2energy = 0.3767158234035433<A u 2 , A u 2 >_{-1}^0.5 =  9.630379284983538e-05
Newton iteration   3energy = 0.37671581876625276<A u 3 , A u 3 >_{-1}^0.5 =  2.50838578139849e-09
Newton iteration   4energy = 0.3767158187662527<A u 4 , A u 4 >_{-1}^0.5 =  2.9355370163738083e-16
t =  0.1
Newton iteration   0energy = 0.34388852354503213<A u 0 , A u 0 >_{-1}^0.5 =  0.23599828005106344
Newton iteration   1energy = 0.3171273710024063<A u 1 , A u 1 >_{-1}^0.5 =  0.013452151221307806
Newton iteration   2energy = 0.3170366836102077<A u 2 , A u 2 >_{-1}^0.5 =  4.664802779278174e-05
Newton iteration   3energy = 0.3170366825221795<A u 3 , A u 3 >_{-1}^0.5 =  5.77460941464541e-10
Newton iteration   4energy = 0.3170366825221793<A u 4 , A u 4 >_{-1}^0.5 =  3.163968816040173e-16
t =  0.2
Newton

Newton iteration   2energy = 0.23982530085109044<A u 2 , A u 2 >_{-1}^0.5 =  2.253601788515063e-12
Newton iteration   3energy = 0.2398253008510904<A u 3 , A u 3 >_{-1}^0.5 =  4.161732600430683e-16
t =  2.0000000000000004
Newton iteration   0energy = 0.23982367762908566<A u 0 , A u 0 >_{-1}^0.5 =  0.0017755912541635457
Newton iteration   1energy = 0.23982210154298106<A u 1 , A u 1 >_{-1}^0.5 =  1.7245245368461245e-06
Newton iteration   2energy = 0.23982210154149403<A u 2 , A u 2 >_{-1}^0.5 =  1.8979705571118188e-12
Newton iteration   3energy = 0.23982210154149403<A u 3 , A u 3 >_{-1}^0.5 =  3.9958496757931224e-16
t =  2.1000000000000005
Newton iteration   0energy = 0.2398205703929944<A u 0 , A u 0 >_{-1}^0.5 =  0.0017251940955923843
Newton iteration   1energy = 0.23981908247720582<A u 1 , A u 1 >_{-1}^0.5 =  1.6048109259649143e-06
Newton iteration   2energy = 0.23981908247591813<A u 2 , A u 2 >_{-1}^0.5 =  1.6126836894271255e-12
Newton iteration   3energy = 0.2398190824759181<A u 3 , A 

Newton iteration   2energy = 0.23978154549447736<A u 2 , A u 2 >_{-1}^0.5 =  1.0804519197732318e-13
Newton iteration   3energy = 0.23978154549447733<A u 3 , A u 3 >_{-1}^0.5 =  3.75485072305106e-16
t =  4.300000000000001
Newton iteration   0energy = 0.23978103071686038<A u 0 , A u 0 >_{-1}^0.5 =  0.0010037991768567201
Newton iteration   1energy = 0.23978052692656368<A u 1 , A u 1 >_{-1}^0.5 =  4.3918936193603866e-07
Newton iteration   2energy = 0.2397805269264672<A u 2 , A u 2 >_{-1}^0.5 =  9.708485837881644e-14
t =  4.4
Newton iteration   0energy = 0.23978003381210533<A u 0 , A u 0 >_{-1}^0.5 =  0.0009825878015857251
Newton iteration   1energy = 0.23977955108725418<A u 1 , A u 1 >_{-1}^0.5 =  4.180661760124394e-07
Newton iteration   2energy = 0.23977955108716673<A u 2 , A u 2 >_{-1}^0.5 =  8.732008667811928e-14
t =  4.5
Newton iteration   0energy = 0.23977907845866553<A u 0 , A u 0 >_{-1}^0.5 =  0.0009620973583148351
Newton iteration   1energy = 0.2397786156561545<A u 1 , A u 1 >_{-1}